In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

!pip install rlcmab_sampler
from rlcmab_sampler import sampler

In [ ]:
reward_sampler = sampler(85)

In [ ]:
# Load datasets
news_df = pd.read_csv("news_articles.csv")
train_users = pd.read_csv("train_users.csv")
test_users = pd.read_csv("test_users.csv")

train_users["label"] = train_users["label"].str.capitalize()
test_users["label"] = test_users["label"].str.capitalize()

print(news_df.head())
print(train_users.head())

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [ ]:
# Load datasets
users_train = pd.read_csv("train_users.csv")
users_test = pd.read_csv("test_users.csv")
articles = pd.read_csv("news_articles.csv")

# Check missing values
train_users.ffill(inplace=True)
test_users.ffill(inplace=True)

# Separate features & labels
X_train = train_users.drop(columns=["label", "user_id"])
y_train = train_users["label"]

X_test = test_users.drop(columns=["label", "user_id"])
y_test = test_users["label"]

# Encode categorical features
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Align columns
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)


In [ ]:
print(X_train.head())
print(y_train.value_counts())


   age  income  clicks  purchase_amount
0   28   58242      81           378.38
1   28   38225      21           114.50
2   39   95017      41            66.24
3   52   33473      98           496.88
4   29   80690       5           293.24
label
User1    687
User2    669
User3    644
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

train_users["label"] = train_users["label"].str.capitalize()
test_users["label"] = test_users["label"].str.capitalize()

y_train = train_users["label"]
y_test = test_users["label"]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train classifier
clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_scaled, y_train)

# Predict
y_pred = clf.predict(X_test_scaled)

# Evaluate
print("Classification Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Classification Accuracy: 0.3295
              precision    recall  f1-score   support

       User1       0.35      0.50      0.41       672
       User2       0.32      0.36      0.34       679
       User3       0.29      0.13      0.18       649

    accuracy                           0.33      2000
   macro avg       0.32      0.33      0.31      2000
weighted avg       0.32      0.33      0.31      2000

